---



# Make C-Star Cases

This notebook takes the input datasets prepared with `roms-tools` in the previous notebook and prepares and exports two C-Star Cases.
The first uses netCDF input datasets (and can thus be prepared on any machine)
The second uses yaml files to be passed to roms-tools via C-Star.

---
## Common steps (both netcdf and yaml):

In [1]:
# If `local=True`, the blueprints will use local filesystem paths for AdditionalCode and InputDataset sources.
# Otherwise, the blueprints will use URLs to repositories and files
local=False
if local:
    code_location = str(Path.cwd())+"/"
    additional_code_checkout_target = None
    netcdf_dataset_location = str(Path("input_datasets_netcdf/").resolve())+"/"
    yaml_dataset_location = str(Path("input_datasets_yaml/").resolve())+"/"
else:
    additional_code_location = "https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git"
    additional_code_checkout_target = "cstar_alpha"
    netcdf_dataset_location = "https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example/raw/netcdf_inputs/input_datasets_netcdf/"
    yaml_dataset_location = "https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example/raw/cstar_alpha/input_datasets_yaml/"

### BaseModel

In [4]:
from cstar.roms import ROMSBaseModel
from cstar.marbl import MARBLBaseModel

roms_base_model = ROMSBaseModel(
    source_repo='https://github.com/CESR-lab/ucla-roms.git',
    checkout_target="52adceac25670ee985d741d1bad62639dac3d183",
)

marbl_base_model = MARBLBaseModel(
    source_repo='https://github.com/marbl-ecosys/MARBL.git',
    checkout_target='marbl0.45.0'
)

### MARBLComponent

In [13]:
from cstar.marbl import MARBLComponent

marbl_component = MARBLComponent(
    base_model = marbl_base_model
)
print(marbl_component)

MARBLComponent
--------------
base_model: MARBLBaseModel instance (query using Component.base_model)


### AdditionalCode

In [5]:

from cstar.base import AdditionalCode

roms_namelists = AdditionalCode(
    location = additional_code_location,
    subdir = "roms_runtime_code",
    checkout_target = additional_code_checkout_target    ,
    files = [
        "roms.in_TEMPLATE",
        "marbl_in",
        "marbl_tracer_output_list",
        "marbl_diagnostic_output_list"
    ]
)
print(roms_namelists)

AdditionalCode
--------------
Location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
subdirectory: roms_runtime_code
Working path: None
Exists locally: False (get with AdditionalCode.get())
Files:
    roms.in_TEMPLATE      (roms.in will be used by C-Star based on this template)
    marbl_in
    marbl_tracer_output_list
    marbl_diagnostic_output_list


In [6]:
roms_additional_source_code = AdditionalCode(
    location = additional_code_location,
    subdir = "roms_compile_time_code",
    checkout_target = additional_code_checkout_target,
    files = [
        "bgc.opt",
         "bulk_frc.opt",
         "cppdefs.opt",
         "diagnostics.opt",
         "ocean_vars.opt",
         "param.opt",
         "tracers.opt",
         "Makefile",
         "Make.depend",
    ]
)

print(roms_additional_source_code)

AdditionalCode
--------------
Location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
subdirectory: roms_compile_time_code
Working path: None
Exists locally: False (get with AdditionalCode.get())
Files:
    bgc.opt
    bulk_frc.opt
    cppdefs.opt
    diagnostics.opt
    ocean_vars.opt
    param.opt
    tracers.opt
    Makefile
    Make.depend


### Discretization

In [11]:
from cstar.roms import ROMSDiscretization

roms_discretization = ROMSDiscretization(time_step = 720,
                                         n_procs_x = 3,
                                         n_procs_y = 3)
print(roms_discretization)

ROMSDiscretization
------------------
time_step: 720s
n_procs_x: 3 (Number of x-direction processors)
n_procs_y: 3 (Number of y-direction processors)


---
## NetCDF-specific steps

### InputDatasets (netcdf)

In [7]:
%%bash
if [ -d input_datasets_netcdf ];then
    cd input_datasets_netcdf/
    shasum -a 256 *.nc
fi

In [8]:
from cstar.roms import ROMSModelGrid, ROMSInitialConditions, ROMSTidalForcing, ROMSBoundaryForcing, ROMSSurfaceForcing

# Boundary
roms_phys_boundary_forcing_nc = ROMSBoundaryForcing(
    location=netcdf_dataset_location + "roms_bry.nc",
    file_hash="93dc9e8aaebecec61895e344898a7679d058b8c8768436c45e7cfe6d672d5380",
)
roms_bgc_boundary_forcing_nc = ROMSBoundaryForcing(
    location = netcdf_dataset_location+"roms_bry_bgc.nc",
    file_hash = "1b2f1cc83441cc80d45055985a01bfaa0bc698104a2c58a00e7a87fafc214575",
)

# Surface
roms_phys_surface_forcing_nc = ROMSSurfaceForcing(
    location=netcdf_dataset_location + "roms_frc.nc",
    file_hash="8ef325c9242db183d27bf037b91c57d42fa04d9bb7ec7249fecc53ca707df2fb",
)
roms_bgc_surface_forcing_nc = ROMSSurfaceForcing(
    location=netcdf_dataset_location + "roms_frc_bgc.nc",
    file_hash="43bd29ce46394fd62a9f6da299e8cb414df4b21ccf48c64830ee0f560252af39",
)

#Grid
roms_model_grid_nc = ROMSModelGrid(
    location=netcdf_dataset_location + "roms_grd.nc",
    file_hash="84f3667f39add10e977bb87c74c40776641e8fb27ad5d590948c858afe84a045",
)

# Initial conditions
roms_initial_conditions_nc = ROMSInitialConditions(
    location=netcdf_dataset_location + "roms_ini.nc",
    file_hash="1ffa9a85fe30c4c6999969e716ff6de833fc901d1cc213e89760bff8c22552bd",
)

# Tides
roms_tidal_forcing_nc = ROMSTidalForcing(
    location=netcdf_dataset_location + "roms_tides.nc",
    file_hash="a711d31087d2b0ed6569736db5139d17100c041d03484892e241a624eb4f9ea9",
)



### ROMSComponent (netcdf)

In [14]:
from cstar.roms import ROMSComponent
roms_component_nc = ROMSComponent(
    base_model = roms_base_model,
    namelists = roms_namelists,
    additional_source_code = roms_additional_source_code,
    discretization = roms_discretization,
    model_grid = roms_model_grid_nc,
    initial_conditions = roms_initial_conditions_nc,
    tidal_forcing = roms_tidal_forcing_nc,
    boundary_forcing = [roms_phys_boundary_forcing_nc,roms_bgc_boundary_forcing_nc],
    surface_forcing = [roms_phys_surface_forcing_nc, roms_bgc_surface_forcing_nc]
)
print(roms_component_nc)



ROMSComponent
-------------
base_model: ROMSBaseModel instance (query using Component.base_model)
additional_source_code: AdditionalCode instance with 9 files (query using Component.additional_source_code)
namelists: AdditionalCode instance with 4 files (query using Component.namelists)
model_grid = <ROMSModelGrid instance>
initial_conditions = <ROMSInitialConditions instance>
tidal_forcing = <ROMSTidalForcing instance>
surface_forcing = <list of 2 ROMSSurfaceForcing instances>
boundary_forcing = <list of 2 ROMSBoundaryForcing instances>

Discretization:
ROMSDiscretization
------------------
time_step: 720s
n_procs_x: 3 (Number of x-direction processors)
n_procs_y: 3 (Number of y-direction processors)


### Case (netcdf)

In [16]:
from cstar import Case

In [18]:
roms_marbl_case_nc = Case(
    components=[marbl_component, roms_component_nc],
    name='roms_marbl_example_cstar_case_nc',
    start_date = "20120101 12:00:00",
    end_date = "20120110 12:00:00",
    caseroot = "roms_marbl_example_cstar_case_netcdf",
    valid_start_date = "20120101 12:00:00",
    valid_end_date = "20120131 12:00:00"
)
print(roms_marbl_case_nc)
roms_marbl_case_nc.to_blueprint("cstar_blueprint_example_with_netcdf_inputs.yaml")

C-Star Case
-----------
Name: roms_marbl_example_cstar_case_nc
caseroot: /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/blueprints/cstar_blueprint_roms_marbl_example/roms_marbl_example_cstar_case_netcdf
start_date: 2012-01-01 12:00:00
end_date: 2012-01-10 12:00:00
Is setup: False
Valid date range:
valid_start_date: 2012-01-01 12:00:00
valid_end_date: 2012-01-31 12:00:00

It is built from the following Components (query using Case.components): 
   <MARBLComponent instance>
   <ROMSComponent instance>


### Setup and run (netcdf)

In [ ]:
roms_marbl_case_nc.setup()
roms_marbl_case_nc.build()
roms_marbl_case_nc.pre_run()

In [ ]:
lp=roms_marbl_case_nc.run()

In [ ]:
lp.updates(seconds=0,confirm_indefinite=False)

In [ ]:
roms_marbl_case_nc.post_run()

---
## YAML-specific steps

### InputDatasets (yaml)

In [9]:
%%bash
if [ -d input_datasets_yaml ];then
    cd input_datasets_yaml/
    shasum -a 256 *.yaml
fi

8c144be6ef23e3b91beb37ae9bdd52f582906729f9fd34849838d7ac4d05af36  roms_bry_bgc.yaml
857db7b4ce0daf9bfe12a60a887645c49656c4d585286ca0c3715046fc8f3568  roms_bry.yaml
2858a657b086a24764d25957b2d39893450321d6ed89ce7c88307e9f4eea2341  roms_frc_bgc.yaml
b529ba66a6856a040bb3a21f90a2b4f2d62ecb2cd747ba81d0253d1b157846a9  roms_frc.yaml
00f5cfed8eed278f9eb3c207158ab7d3ecec8c3c5bbf39265ace25a9492a44f9  roms_grd.yaml
1d60e298e428a7c6dc399464e00e72cf755416b4b1b0bd9f754a08fac2d73da6  roms_ini.yaml
8376a467ffef06d97af842ef602453a24d810086e7c3f75483d3a38cd0594d83  roms_riv_frc.yaml
17e82347efab35e6508a0671a698bae8e93e1e944fc9afdb03ed19e3e363c7c4  roms_tides.yaml


In [10]:
from cstar.roms import ROMSModelGrid, ROMSInitialConditions, ROMSTidalForcing, ROMSBoundaryForcing, ROMSSurfaceForcing

# Boundary
roms_bgc_boundary_forcing_yaml = ROMSBoundaryForcing(
    location = yaml_dataset_location + "roms_bry_bgc.yaml",
    file_hash = "8c144be6ef23e3b91beb37ae9bdd52f582906729f9fd34849838d7ac4d05af36",
)
roms_phys_boundary_forcing_yaml = ROMSBoundaryForcing(
    location=yaml_dataset_location + "roms_bry.yaml",
    file_hash="857db7b4ce0daf9bfe12a60a887645c49656c4d585286ca0c3715046fc8f3568",
)


# Surface
roms_bgc_surface_forcing_yaml = ROMSSurfaceForcing(
    location=yaml_dataset_location + "roms_frc_bgc.yaml",
    file_hash="2858a657b086a24764d25957b2d39893450321d6ed89ce7c88307e9f4eea2341",
)
roms_phys_surface_forcing_yaml = ROMSSurfaceForcing(
    location=yaml_dataset_location + "roms_frc.yaml",
    file_hash="b529ba66a6856a040bb3a21f90a2b4f2d62ecb2cd747ba81d0253d1b157846a9",
)

#Grid
roms_model_grid_yaml = ROMSModelGrid(
    location=yaml_dataset_location + "roms_grd.yaml",
    file_hash="00f5cfed8eed278f9eb3c207158ab7d3ecec8c3c5bbf39265ace25a9492a44f9",
)

# Initial conditions
roms_initial_conditions_yaml = ROMSInitialConditions(
    location=yaml_dataset_location + "roms_ini.yaml",
    file_hash="1d60e298e428a7c6dc399464e00e72cf755416b4b1b0bd9f754a08fac2d73da6",
)

# Tides
roms_tidal_forcing_yaml = ROMSTidalForcing(
    location=yaml_dataset_location + "roms_tides.yaml",
    file_hash="17e82347efab35e6508a0671a698bae8e93e1e944fc9afdb03ed19e3e363c7c4",
)



### ROMSComponent (yaml)

In [15]:
from cstar.roms import ROMSComponent
roms_component_yaml = ROMSComponent(
    base_model = roms_base_model,
    namelists = roms_namelists,
    additional_source_code = roms_additional_source_code,
    discretization = roms_discretization,
    model_grid = roms_model_grid_yaml,
    initial_conditions = roms_initial_conditions_yaml,
    tidal_forcing = roms_tidal_forcing_yaml,
    boundary_forcing = [roms_phys_boundary_forcing_yaml,roms_bgc_boundary_forcing_yaml],
    surface_forcing = [roms_phys_surface_forcing_yaml, roms_bgc_surface_forcing_yaml]
)
print(roms_component_yaml)



ROMSComponent
-------------
base_model: ROMSBaseModel instance (query using Component.base_model)
additional_source_code: AdditionalCode instance with 9 files (query using Component.additional_source_code)
namelists: AdditionalCode instance with 4 files (query using Component.namelists)
model_grid = <ROMSModelGrid instance>
initial_conditions = <ROMSInitialConditions instance>
tidal_forcing = <ROMSTidalForcing instance>
surface_forcing = <list of 2 ROMSSurfaceForcing instances>
boundary_forcing = <list of 2 ROMSBoundaryForcing instances>

Discretization:
ROMSDiscretization
------------------
time_step: 720s
n_procs_x: 3 (Number of x-direction processors)
n_procs_y: 3 (Number of y-direction processors)


### Case (yaml)

In [19]:
roms_marbl_case_yaml = Case(
    components=[marbl_component, roms_component_yaml],
    name='roms_marbl_example_cstar_case_yaml',
    start_date = "20120101 12:00:00",
    end_date = "20120110 12:00:00",
    caseroot = "roms_marbl_example_cstar_case_yaml",
    valid_start_date = "20120101 12:00:00",
    valid_end_date = "20120131 12:00:00"
)
print(roms_marbl_case_yaml)
roms_marbl_case_yaml.to_blueprint("cstar_blueprint_example_without_prepared_inputs.yaml")

C-Star Case
-----------
Name: roms_marbl_example_cstar_case_yaml
caseroot: /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/blueprints/cstar_blueprint_roms_marbl_example/roms_marbl_example_cstar_case_yaml
start_date: 2012-01-01 12:00:00
end_date: 2012-01-10 12:00:00
Is setup: False
Valid date range:
valid_start_date: 2012-01-01 12:00:00
valid_end_date: 2012-01-31 12:00:00

It is built from the following Components (query using Case.components): 
   <MARBLComponent instance>
   <ROMSComponent instance>


### Setup and run (yaml)

In [ ]:
roms_marbl_case_yaml.setup()
roms_marbl_case_yaml.build()
roms_marbl_case_yaml.pre_run()

In [88]:
import os

In [91]:
lp_yaml=roms_marbl_case_yaml.run(account_key = os.environ.get("ACCOUNT_KEY"),queue_name="shared",walltime="00:10:00")


Running ROMS... 


In [95]:
lp_yaml.updates(seconds=0,confirm_indefinite=False)

This job is currently not running (pending). Live updates cannot be provided.


In [ ]:
roms_marbl_case_yaml.post_run()